In [5]:
!curl https://topcs.blob.core.windows.net/public/FlightData.csv -o flightdata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file flightdata.csv: No such file or directory
  1 1552k    1 15978    0     0  52215      0  0:00:30 --:--:--  0:00:30 53979
curl: (23) Failed writing body (0 != 15978)


In [7]:
import pandas as pd

df = pd.read_csv('FlightData.csv')
df.head()

EmptyDataError: No columns to parse from file

In [ ]:
df.shape

In [ ]:
df.isnull().values.any()

In [ ]:
df.isnull().sum()

In [ ]:
# this column was mistakenly created because of an extra column in the dataset
df = df.drop('Unnamed: 25', axis=1)
df.isnull().sum()

In [ ]:
# df = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15", "CANCELLED", "DIVERTED"]]
df = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]]

df.isnull().sum()

In [ ]:
df[df.isnull().values.any(axis=1)].head()

In [ ]:
# business decision to mark flights as late if flight was cancelled or diverted
df = df.fillna({'ARR_DEL15': 1})
df.iloc[177:185]

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
# we implement quantization by dividing the scheduled departure time
# by 100 to get 24 distinct values, since it matters more if a flight leaves
# at 5pm rather than at 10:30am, instead of 10:40am from 10:30am

In [ ]:
import math
for index, row in df.iterrows():
    df.loc[index, 'CRS_DEP_TIME'] = math.floor(row['CRS_DEP_TIME'] / 100) 
    
df.head()

In [ ]:
df = pd.get_dummies(df, columns=['ORIGIN', 'DEST'])
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df.drop('ARR_DEL15', axis=1), df['ARR_DEL15'], test_size=0.2, random_state=42)

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
train_y.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=13)
model.fit(train_x, train_y)